Hello, i'm currently a graduate student at Gyeongsang National University in Korea.  
I'm going to make a Corona Data dashboard on this laptop.

import packages

In [ ]:
!pip install dash

In [ ]:
!pip install dash_deck

In [ ]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from datetime import datetime as dt
import shapely
from shapely.ops import unary_union
import geopandas as gpd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_deck
import plotly.express as px
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import GRU, Dense


I used mapping data(map: shapefile  source_website : https://gadm.org/index.html)  

and corona data(I can't remember where I got this data from. If anyone knows where the data is from, please tell me!)

In [ ]:
# #read shapefile
# gdf = gpd.read_file('gadm36.shp')
# name_lst = []
# geo_lst = []

# for i in tqdm(gdf.NAME_0. unique()):
#     name_lst.append(i)
#     geo_lst.append(unary_union(gdf[gdf.NAME_0==i]['geometry']))

When I run the code above, it takes about an hour on my computer. The shapefile is over 1GB.  
So, I will use the data set I cleaned up on my computer. I'll just show you the process of sorting it out.

In [ ]:
# Show plot(read shapefile)
# gdf.plot()

In [ ]:
# make geodataframe(only using columns)
# geo_df = gpd.GeoDataFrame()
# geo_df['country'] = name_lst
# geo_df['geometry'] = geo_lst
# geo_df

The geodataframe is still large, so extract the centeroid and save it

In [ ]:
# lst = []
# for i in geo_df.geometry:
#     lst.append(i.centroid)

In [ ]:
# geo_df['center'] = lst
# geo_df

In [ ]:
# copy
# geo_df2 = pd.DataFrame(geo_df).copy()

In [ ]:
# delete geometry
# del geo_df2['geometry']
# geo_df2

In [ ]:
# Stored by latitude and longitude
# geo_df2['lat'] = geo_df2['center'].apply(lambda coord: coord.y)
# geo_df2['lng'] = geo_df2['center'].apply(lambda coord: coord.x)
# del geo_df2['center']
# geo_df2

Finish shapefile preprocessing

In [ ]:
# read_csv
geo_df2 = pd.read_csv('../input/dataset/geodata.csv')

In [ ]:
geo_df2.head()

In [ ]:
geo_df2.index = list(geo_df2.iloc[:,0])
geo_df2 = geo_df2.iloc[:,1:]
geo_df2.head()

Read corona data

In [ ]:
df = pd.read_csv('../input/dataset2/owid-covid-data.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.info()

Changed to datetime type because date data is stored as object.

In [ ]:
lst = []
for i in df.date:
    lst.append(dt.strptime(i,'%Y-%m-%d').date())
df.loc[:,'date'] = lst

Check if the country name in both dataframes is the same.(df.location, geo_df2.country)

In [ ]:
set(df.location) - set(geo_df2.country)

In [ ]:
set(geo_df2.country) - set(df.location)

delete country and save continent(Africa, Asia, Europe, European Union, International, North America, Oceania, South America, World

In [ ]:
delete_country = set(df.location) - set(geo_df2.country) - set(['Africa','Asia','Europe','European Union','International','North America', 'Oceania','South America','World'])

In [ ]:
delete_country = list(delete_country)

In [ ]:
lst = []
for i in delete_country:
    lst.extend(df[df.location == i].index)

In [ ]:
df = df.drop(lst,axis=0)

Those with missing values in continent are the values of location we left out. Therefore, we add a continet to the continent.

In [ ]:
df[df.location=='Africa'].head()

In [ ]:
df.continent = df.continent.fillna('continent')

Plotly image load

In [ ]:
from PIL import Image
img = Image.open('../input/dataset3/dash-logo.png')
img

Expect Corona 10days(use Gru)

In [ ]:
gru_model = keras.Sequential()
gru_model.add(GRU(units=30,input_shape=(10,1),activation='relu',return_sequences=True))
gru_model.add(GRU(10,activation='relu'))
gru_model.add(Dense(10))
gru_model.compile(loss='mse',optimizer='adam')
gru_model.summary()

In [ ]:
## data split(used Gru)
def split_x(seq,size):
    aaa = []
    for i in range(len(seq) - size +1):
        subset = seq[i:(i+size)]
        aaa.append([item for item in subset])
    return np.array(aaa)

In [ ]:
### Running dashboard Gru
def gru_learning(country):
    dff = df[df.location==country]['new_cases']
    ## if new_cases have none, input zero
    if len(dff[dff.isna()==True]) == 0: 
        dataset = split_x(dff,20)
    else:
        dff = dff.fillna(0)
        dataset = split_x(dff,20)
    # input data(model)(preprocessing)
    train_x = dataset[:,:-10]
    train_y = dataset[:,-10:]
    split_num = round(train_x.shape[0] * 0.7)
    valid_x = train_x[split_num:,:]
    valid_y = train_y[split_num:,]
    train_x = train_x[:split_num,:]
    train_y = train_y[:split_num,]
    train_x = train_x.reshape((train_x.shape[0],train_x.shape[1],1))
    valid_x = valid_x.reshape((valid_x.shape[0],valid_x.shape[1],1))
    # gru training
    history = gru_model.fit(train_x,train_y, epochs = 10, batch_size = 12, validation_data=(valid_x,valid_y))
    test = np.array(dff[-10:]).reshape(1,10,1)
    result = gru_model.predict(test)
    return result ### if you want history, append reture value history

I use mapbox, so i load mapbox api token.  
If you want to receive tokens, you can sign up for mapbox.  https://www.mapbox.com/

In [ ]:
# mapbox api
mapbox_api='pk.eyJ1Ijoia2ltc3VuZ2RvbiIsImEiOiJja296YzVpZTAwYzFjMnBuenl1cTBmdjJuIn0.2zzoZdXYU5hgL45PPg-hLg'

In [ ]:
# Initial map drawing

dff = df[['date','location','total_cases','new_cases']]

dff = dff[dff.date==dt(2020,2,24).date()]

dff2 = pd.merge(geo_df2,dff,how='left',left_on='country',right_on='location')

dff2 = dff2.dropna(axis=0)

px.set_mapbox_access_token(mapbox_api)

fig = px.scatter_mapbox(dff2, lat="lat", lon="lng", color="total_cases", size="new_cases",template='plotly_dark',text='country',title="marker size = new_cases, marker color = total_cases",
              color_continuous_scale='ylorbr', size_max=30, zoom=3,mapbox_style='dark')

Now let's create the dashboard.

In [ ]:
##### using label ######
# continent
list_of_continent = df.continent.unique()
# country
list_of_country = df.location.unique()
# y-axis value to use for time series graph label
list_of_columns = ['total_cases','new_cases','new_cases_smoothed','total_deaths','new_deaths','new_deaths_smoothed','total_cases_per_million','new_cases_per_million','new_cases_smoothed_per_million','total_deaths_per_million','new_deaths_per_million','new_deaths_smoothed_per_million','reproduction_rate']
# Basic summary information by country
list_of_table=['stringency_index','population','population_density','median_age','aged_65_older','aged_70_older','gdp_per_capita','cardiovasc_death_rate','diabetes_prevalence','hospital_beds_per_thousand','life_expectancy','human_development_index']

In [ ]:
## make dashboard

app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.Div([html.H1(children = 'Corona Dashboard')
                 ]),
        html.Div([dcc.Markdown(["Let's check the trend of corona confirmed by using global corona data"])
                 ]),
        html.Div([dcc.Markdown(['### select date']),
                 dcc.DatePickerSingle(id = 'date-picker',
                                     min_date_allowed = dt(2020,2,20),
                                     max_date_allowed = dt(2021,5,20),
                                     initial_visible_month = dt(2020,2,24),
                                     date = dt(2020,2,24).date(),
                                     display_format = 'YYYY-MM-DD',
                                     style = {'border': '0px solid black'},)
                 ]),
        html.Div([dcc.Markdown(['### select Continent']),
                 dcc.Dropdown(id = 'continent=picker',
                             options = [{'label':i,'value':i} for i in list_of_continent],value = 'continent',
                             style = {'backgroundColor':'white','color':'black'})
                 ]),
        html.Div([dcc.Markdown(['### select Country']),
                 dcc.Dropdown(id = 'country-picker',
                             options=[{'label':i,'value':i} for i in list_of_country],value='World',
                             style = {'border':'0px solid black','backgroundColor':'white','color':'black'})
                 ]),
        html.Div(dcc.Markdown(id = 'date-country-markdown'), style = {'backgroundColor':'black','color':'white'}),
        html.Div([dcc.Markdown('## =========================')]),
        html.Div([dcc.Markdown('## data source')]),
        html.Div([dcc.Markdown('### corona data : https://coronavirus.jhu.edu/map.html')]),
        html.Div([dcc.Markdown('### geomap data : https://gadm.org/index.html')]),
        html.Div([html.Img(className='logo',src=img)]),
        html.Div([dcc.Markdown('# GNU stat')])
    ], style = {'width':'20%','display':'inline-block','backgroundColor':'black'}),
    html.Div([
        html.Div([html.H1(children = 'World Map')]),
        html.Div(dcc.Graph(id = 'world-map',figure=fig)),
        html.Div([dcc.Markdown('## column select(y-axis)'),
                 dcc.Dropdown(id = 'column-select',
                             options=[{'label':i,'value':i} for i in list_of_columns],value='total_cases',
                             style = {'backgroundColor':'gray','color':'black'}),
                  dcc.Graph(id='country-timeseries-plot')],style={'backgroundColor':'black'}),
        dcc.Loading(html.Div([dcc.Graph(id = 'expected(Gru)~10days')]))
                 ], style= {'width':'79%','float':'right','display':'inline-block','backgroundColor':'black'}),
    ],style = {'backgroundColor':'#3A3939','color':'white'})

########################################## callback ###############################################

# Above, I took the form of the web page, gave an id to each column, and set the style.

# In the lower part, a function called callback is used to interact with each cell.

########################################## callback ###############################################

##callback Description
# Use @app.callback
# Put the output callback result values into the properties in the Output
# Specify the data to be used to derive the input callback result.
# Put the return value in the output through the operation of def.

@app.callback(Output('country-picker','options'),
             Input('continent-picker','value'))

def country_selector(choose_continent):
    dff = df[df.continent==choose_continent]
    return [{'label':i,'value':i} for i in dff.location.unique()]

@app.callback(Output('world-map','figure'),
             [Input('date-picker','date')])

def mapping_corona(datePicked):
    datePicked = dt.strptime(datePicked, '%Y-%m-%d').date()
    dff = df[['date','location','total_cases','new_cases']]
    dff = dff[dff.date == datePicked]
    dff2 = pd.merge(geo_df2,dff, how='left',left_on='country',right_on = 'location')
    dff2 = dff2.dropna(axis = 0)
    px.set_mapbox_access_token(mapbox_api)
    
    # scatter mapping
    # Latitude and longitude of lat, lon data
    # color = we will use the total_cases value to set the color of the points
    # size = new_cases will be used to set the size of the points.
    # template = set the shape of the graph
    # text = Show the country (country name) in the interactive box that appears when you move the mouse cursor over the dot.
    # color_continuous_scale = Set the color of the point (you need to use a continuous color) to see what colors can be used on top
    # size_max = set the maximum size of the point (there is no size_min, but if size_max is increased, size_min seems to increase)
    # height = set graph size
    # mapbox_style = set map layer
    fig = px.scatter_mapbox(dff2, lat="lat", lon="lng", color="total_cases", size="new_cases",template='plotly_dark',text='country',title="marker size = new_cases, marker color = total_cases",
              color_continuous_scale='ylorbr', size_max=30, zoom=1,height=600,mapbox_style='dark')
    return fig

@app.callback(Output('country-timeseries-plot','figure'),
             [Input('continent-picker','value'),
             Input('country-picker','value'),
             Input('column-select','value')])

def create_country_timeseries(continent,country,select_column):
    dff = df[df.continent==continent]
    dff = dff[dff.location==country]
    dff = dff[['date',select_column]]
    fig = px.scatter(dff, x='date', y = select_column, template = 'plotly_dark', title = country + ' :'+select_column)
    fig.update_traces(mode='lines')
    fig.update_xaxes(showgrid=False)
    fig.update_layout(height = 300,margin = {'l':0,'b':0,'r':0,'t':30})
    return fig

@app.callback(Output('date-country-markdown','children'),
             [Input('country-picker','value')])

def update_markdown(country):
    dff = df[df.location == country]
    lst = []
    for i in list_of_table:
        idx = dff.index[0]
        if dff[i].isna()[idx] == True:
            lst.append('None')
        else:
            lst.append(dff[i][idx])
        
    sentence = '''
    #### Select Country Information
    #### stringency_index : {0}
    #### population : {1}
    #### population_density : {2}
    #### median_age : {3}
    #### aged_65_older : {4}
    #### aged_70_older : {5}
    #### gdp_per_capita : {6}
    #### cardiovasc_death_rate : {7}
    #### diabetes_prevalence : {8}
    #### hospital_beds_per_thousand : {9}
    #### life_expectancy : {10}
    #### human_development_index : {11}
    '''.format(str(lst[0]),str(lst[1]),str(lst[2]),str(lst[3]),str(lst[4]),str(lst[5]),str(lst[6]),str(lst[7]),str(lst[8]),str(lst[9]),str(lst[10]),str(lst[11]))
    return sentence

@app.callback(Output('expected(Gru)~10days','figure'),
             [Input('country-picker','value')])

def create_gru_prediction(country):
    result = gru_learning(country)
    dff = pd.DataFrame(result).T
    lst = [1,2,3,4,5,6,7,8,9,10]
    dff['date'] = lst
    dff.columns = ['new_cases','date']
    fig = px.scatter(dff, x='date',y='new_cases',template='plotly_dark',title = country + ' predict 10days')
    fig.update_traces(mode = 'lines')
    fig.update_xaxes(showgrid=False)
    fig.update_layout(height = 300, margin = {'l':0,'b':0,'r':0,'t':30})
    return fig
if __name__ == '__main__':
    app.run_server(debug=False)

I don't know how to upload the results dashboard, so I'll just write the code.